In [1]:
import json

In [2]:
with open("dataset.json", "r") as f:
    datasets = json.load(f)

In [3]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
from datasets import Dataset

In [4]:
model_name = "rinna/japanese-gpt2-medium"

# モデルの選択

rinna/japanese-gpt2-medium
にしたのは、
rinna/japanese-gpt-1b
だと、GPUのスペック不足でtrain()が進行しなかったため。


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/806k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

In [7]:
def tokenize_function(examples):
    inputs = tokenizer(examples['input'], padding="max_length", truncation=True, max_length=7)
    labels = tokenizer(examples['output'], padding="max_length", truncation=True, max_length=7)
    inputs['labels'] = labels['input_ids']
    return inputs

In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs'
)

In [9]:
train_eval_border_index = round(len(datasets) * 0.8)

In [10]:
train_dataset = Dataset.from_list(datasets[:train_eval_border_index]).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_list(datasets[train_eval_border_index:]).map(tokenize_function, batched=True)

Map:   0%|          | 0/2460 [00:00<?, ? examples/s]

Map:   0%|          | 0/615 [00:00<?, ? examples/s]

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
500,2.539700
1000,1.516800
